Recipe Suggestion Agentic Chatbot

In [41]:
%%capture --no-stderr
%pip install langgraph langgraph-checkpoint-sqlite langchain_core langchain-google-genai

In [42]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")

In [43]:
import os

os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

In [44]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(api_key=GEMINI_API_KEY, model="gemini-1.5-flash")

In [45]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
import sqlite3

file_path = '/content/drive/My Drive/state_db/example.db'
conn = sqlite3.connect(file_path, check_same_thread=False)

In [47]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory: SqliteSaver = SqliteSaver(conn)

In [ ]:
from langgraph.graph import MessagesState
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage, ToolMessage, AnyMessage
from langgraph.graph.message import add_messages
from langchain.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import display, Image
from typing import Annotated
from typing_extensions import TypedDict

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    summary: str

@tool
def recipe_search_tool(state: State):
  """Searches for a recipe based on the conversation and returns it as a ToolMessage."""
  messages = state["messages"]

  system_message = SystemMessage(content="You are a recipe assistant. Based on the user's input, suggest recipes from https://www.allrecipes.com/ that use the provided ingredients.")

  recipe_response = llm.invoke(system_message + messages)

  return {"messages": ToolMessage(content=recipe_response)}

llm_with_tools = llm.bind_tools([recipe_search_tool])

In [ ]:
def assistant(state: State) -> State:
    summary = state.get("summary", "")

    if summary:
        system_message = f"Summary of conversation earlier: {summary}"
        messages = [SystemMessage(content=system_message)] + state["messages"]
    else:
        messages = state["messages"]

    response = llm_with_tools.invoke(messages)
    return {"messages": response}


def summarize_conversation(state: State) -> State:
    summary = state.get("summary", "")

    if summary:
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"

    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = llm_with_tools.invoke(messages)

    delete_messages = []
    for m in state["messages"][:-2]:
      delete_messages.append(RemoveMessage(id=m.id))

    return {"summary": response.content, "messages": delete_messages}


def should_summarize(state: State) -> State:
    """Return the next node to execute."""

    messages = state["messages"]

    if len(messages) > 6:
        return "summarize_conversation"

    return END

In [ ]:
builder: StateGraph = StateGraph(State)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode([recipe_search_tool]))
builder.add_node(summarize_conversation)

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")
builder.add_conditional_edges("assistant", should_summarize)
builder.add_edge("summarize_conversation", END)

graph: CompiledStateGraph = builder.compile(checkpointer=memory)

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [73]:
from typing import List

food = input("What would you like to make today? ")

ingredients = input(f"Great! Now, please list the ingredients you have available for your {food}, separated by commas. For example: chicken, onions, garlic, tomatoes, pasta").split(",")

prompt = f"I want to make {food} and I have these ingredients at home: {', '.join(ingredients)}"

What would you like to make today? burger
Great! Now, please list the ingredients you have available for your burger, separated by commas. For example: chicken, onions, garlic, tomatoes, pastasauce, sausages, kebab, mushrooms


In [74]:
config = {"configurable": {"thread_id": "1"}}

input_message = HumanMessage(content=prompt, name="User")

output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

That's a creative combination of ingredients for a burger! Here are a couple of options, depending on what kind of burger you're aiming for:

**Option 1: The "Sausage & Kebab" Burger**

This option focuses on the sausages and kebab as the main protein sources.

* **Patty:**  You could either crumble the sausages and kebab meat together to form a patty, or slice the sausages and kebab into thick rounds to layer on the burger.  If crumbling, lightly fry the mixture to help it bind.
* **Toppings:** Sauteed mushrooms add a nice earthy flavor. Add your sauce generously.  Consider adding some cheese if you have any on hand.  A toasted bun completes the burger.

**Option 2: The "Kebab-Stuffed Sausage Burger"**

This option uses the kebab as a filling within the sausage.

* **Patty:** Use the sausages whole, or cut them lengthwise and stuff them with small pieces of the kebab meat.  You can then grill or pan-fry 